In [1]:
import re
import os
import pandas as pd
import numpy as np
import unicodedata
import pickle
from cities_coordinates import CityCoordinator

from _html_parser import HtmlParser, ParsingDataPrepare

pd.set_option('display.max_colwidth', 1000)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_rows', 500)

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
df_country_code = pd.read_csv('csv_files/wikipedia-iso-country-codes.csv') 

In [4]:
df_country_code.columns = [x.lower() for x in df_country_code.columns]
df_country_code['alpha-2 code'] = df_country_code['alpha-2 code'].map(lambda x: str(x).lower())
df_country_code.rename(columns={'english short name lower case':'country', 'alpha-2 code':'iso_code'}, inplace=True)
df_country_code.iso_code = df_country_code.iso_code.str.upper()
df_country_code.shape

(246, 5)

In [5]:
df_country_code.head(3)

,country,iso_code,alpha-3 code,numeric code,iso 3166-2
0,Afghanistan,AF,AFG,4,ISO 3166-2:AF
1,Åland Islands,AX,ALA,248,ISO 3166-2:AX
2,Albania,AL,ALB,8,ISO 3166-2:AL


In [6]:
df_country_code[df_country_code.iso_code.isin(['NAN'])]

,country,iso_code,alpha-3 code,numeric code,iso 3166-2
151,Namibia,NAN,NAM,516,ISO 3166-2:NA


In [7]:
df_country_code.loc[151:151, 'iso_code'] = 'NA'

In [8]:
df_country_code[df_country_code.country.isin(['England', 'Northern Ireland', 'Scotland', 'Wales'])]

,country,iso_code,alpha-3 code,numeric code,iso 3166-2


#### Add file for geocords

In [10]:
# 1 Stop *********************************************************************************

In [11]:
with open('pickle_files/cities_countries/df_for_geo_coords_21_10_2022_bets_coef', 'rb') as f:
    df_for_coords = pickle.load(f)

In [ ]:
# df_for_coords = pd.concat([df_for_coords_1, df_for_coords_2], axis=0, ignore_index=True)

In [12]:
df_for_coords

,game_id,ligue_header,comp_id,season_id,game_utc,goals,ligue,f_team,s_team,pos_f,pts_f,pos_s,pts_s,max_pos,max_pts,1,X,2,ТМ 2.5,ТБ 2.5,first_g,owngoal,mispen,redcard,ct_cards,f_pow,desc,ct_goals,founded,capacity,city,country
0,1458512,Segunda División,707,306,26.03.2021 17:00,0:0,top,Sporting Gijón,Alcorcón,7.0,65.0,17.0,48.0,22.0,82.0,1.95,2.77,4.72,1.33,3.08,None,0.0,0.0,0.0,2.0,Fav,with_events,0,1905,29538,Gijon,Spain
1,1458506,Segunda División,707,306,26.03.2021 19:00,1:3,top,Castellon,Espanyol,21.0,41.0,1.0,82.0,22.0,82.0,6.06,3.64,1.62,1.61,2.28,13,0.0,0.0,1.0,4.0,Pre,with_events,4,1922,14485,Castellon de la Plana,Spain
2,1459331,First League,435,306,26.03.2021 11:00,2:0,sec,Mykolaiv,Alians Lypova Dolyna,4.0,53.0,6.0,51.0,16.0,68.0,1.82,3.42,3.80,1.84,1.90,39,0.0,0.0,1.0,8.0,Fav,with_events,2,1920,25175,Mykolajiv,Ukraine
3,1459330,First League,435,306,26.03.2021 11:00,1:1,sec,Polessya,Krystal Kherson,11.0,35.0,16.0,13.0,16.0,68.0,1.40,3.84,7.70,1.76,1.97,9,0.0,0.0,0.0,6.0,Fav,with_events,2,NaN,11021,Zhytomyr,Ukraine
4,1459334,First League,435,306,26.03.2021 12:00,1:1,sec,Kremin,Hirnyk-Sport,15.0,24.0,9.0,38.0,16.0,68.0,3.08,3.26,2.12,1.73,2.00,22,0.0,0.0,1.0,7.0,Pre,with_events,2,1959,2200,Kremenchuk,Ukraine
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
407100,1728561,Premier League,623,310,28.05.2022 13:00,0:1,thr,Amanat Baghdad,Al Minaa,17.0,35.0,19.0,26.0,20.0,91.0,2.06,3.10,3.66,1.52,2.53,40,0.0,0.0,1.0,0.0,Fav,with_events,1,1957,3000,Baġdād,Iraq
407101,1728563,Premier League,623,310,28.05.2022 15:30,2:1,thr,Arbil,Al Sinaah,11.0,43.0,16.0,35.0,20.0,91.0,NaN,NaN,NaN,NaN,NaN,21,0.0,0.0,0.0,0.0,Nodeterm,with_events,3,1968,40000,Arbil,Iraq
407102,1728564,Premier League,623,310,28.05.2022 15:30,1:1,thr,Naft Maysan,Al Diwaniya,14.0,39.0,18.0,32.0,20.0,91.0,NaN,NaN,NaN,NaN,NaN,2,0.0,0.0,0.0,0.0,Nodeterm,with_events,2,NaN,25000,Amarah,Iraq
407103,1728565,Premier League,623,310,28.05.2022 18:00,3:1,thr,Al Talaba,Samarra FC,2.0,66.0,20.0,11.0,20.0,91.0,1.26,5.15,10.00,1.92,1.88,1,0.0,0.0,0.0,0.0,Fav,with_events,4,1977,40000,Baġdād,Iraq


#### Remove some countries

In [13]:
# Remove USA country because many cities have twice locations in different states, 'Republic of Kosovo' - doesn't have iso_code, 'Syria' - not sure at game's location
print('shape - ', df_for_coords.shape[0])
df_for_coords = df_for_coords[~df_for_coords.country.isin(['USA', 'Republic of Kosovo', 'Syria'])].copy()
print('shape - ', df_for_coords.shape[0])

shape -  407105
shape -  403262


In [ ]:
#  For teams dataframe appoint iso_code:
# ['England', 'Wales', 'Scotland', 'Northern Ireland'] - 'GB'

# For world cities dataframe renames countries:
# 'Tanzania, United Republic of' - Tanzania, 'Moldova, Republic of' - 'Moldova',
# 'Congo, the Democratic Republic of the' - 'Congo DR', 'United States Of America' - 'USA', 'Syrian Arab Republic' - 'Syria',
# 'China' - 'China PR', 'Korea, Republic of (South Korea) - 'Korea Republic', 'Macedonia, the former Yugoslav Republic of' - 'FYR Macedonia', 
# 'Bosnia and Herzegovina' - 'Bosnia-Herzegovina', 'Ireland' - 'Republic of Ireland', 'Libyan Arab Jamahiriya' - 'Libya', 
# 'Côte d'Ivoire' - 'Ivory Coast'

In [14]:
countries_chng = {'Tanzania, United Republic of':'Tanzania', 'Moldova, Republic of':'Moldova',
                  'Congo, the Democratic Republic of the':'Congo DR', 'United States Of America':'USA', 'Syrian Arab Republic':'Syria',
                  'China':'China PR', 'Korea, Republic of (South Korea)':'Korea Republic', 'Macedonia, the former Yugoslav Republic of':'FYR Macedonia', 
                  'Bosnia and Herzegovina':'Bosnia-Herzegovina', 'Ireland':'Republic of Ireland', 'Libyan Arab Jamahiriya':'Libya',
                  'Côte d\'Ivoire':'Ivory Coast', 'Brunei Darussalam':'Brunei'}

In [15]:
df_country_code.country = df_country_code.country.map(lambda x: countries_chng[x] if x in countries_chng.keys() else x)

In [16]:
df_for_coords = df_for_coords.merge(df_country_code[['country', 'iso_code']], how='left', left_on=['country'], right_on=['country'])
df_for_coords.shape

(403262, 33)

In [17]:
df_for_coords[~df_for_coords.country.isin(df_country_code.country.unique())].country.unique()

array(['Spain\xa0\xa0', 'Ukraine\xa0\xa0', 'United Kingdom\xa0\xa0',
       'Australia\xa0\xa0', 'Portugal\xa0\xa0', 'Netherlands\xa0\xa0',
       'Republic of Ireland\xa0\xa0', 'Brazil\xa0\xa0', 'Israel\xa0\xa0',
       'Russia\xa0\xa0', 'Colombia\xa0\xa0', 'Germany\xa0\xa0',
       'Turkey\xa0\xa0', 'Albania\xa0\xa0', 'England', 'France\xa0\xa0',
       'Northern Ireland\xa0\xa0', 'Northern Ireland', 'Scotland',
       'Bulgaria\xa0\xa0', 'Italy\xa0\xa0', 'Belgium\xa0\xa0',
       'Belarus\xa0\xa0', 'Greece\xa0\xa0', 'Austria\xa0\xa0',
       'Romania\xa0\xa0', 'Croatia\xa0\xa0', 'Denmark\xa0\xa0',
       'Switzerland\xa0\xa0', 'Poland\xa0\xa0', 'Czech Republic\xa0\xa0',
       'Serbia\xa0\xa0', 'Georgia\xa0\xa0', 'Mexico\xa0\xa0',
       'Slovakia\xa0\xa0', 'Wales\xa0\xa0', 'Scotland\xa0\xa0',
       'Kazakhstan\xa0\xa0', 'Korea Republic\xa0\xa0',
       'Argentina\xa0\xa0', 'Saudi Arabia\xa0\xa0', 'Cyprus\xa0\xa0',
       'Uruguay\xa0\xa0', 'Armenia\xa0\xa0', 'Iran\xa0\xa0',
      

In [18]:
british_list           = ['England', 'Northern Ireland', 'Scotland', 'Wales']
df_for_coords.iso_code = df_for_coords[['country', 'iso_code']].apply(lambda x: 'GB' if x[0] in british_list else x[1], axis=1)

In [19]:
df_for_coords[df_for_coords.iso_code.isna()].values

array([['1458512', 'Segunda División', '707', ..., 'Gijon',
        'Spain\xa0\xa0', nan],
       ['1458506', 'Segunda División', '707', ...,
        'Castellon de la Plana', 'Spain\xa0\xa0', nan],
       ['1459330', 'First League', '435', ..., 'Zhytomyr',
        'Ukraine\xa0\xa0', nan],
       ...,
       ['1718809', '1. Division', '635', ..., 'Kyzylorda',
        'Kazakhstan\xa0\xa0', nan],
       ['1705744', 'Campeonato Brasileiro C', '527', ...,
        'Florianopolis', 'Brazil\xa0\xa0', nan],
       ['1603939', 'Championnat National 2', '579', ..., 'Metz',
        'France\xa0\xa0', nan]], dtype=object)

In [20]:
df_for_coords.head(3)

,game_id,ligue_header,comp_id,season_id,game_utc,goals,ligue,f_team,s_team,pos_f,pts_f,pos_s,pts_s,max_pos,max_pts,1,X,2,ТМ 2.5,ТБ 2.5,first_g,owngoal,mispen,redcard,ct_cards,f_pow,desc,ct_goals,founded,capacity,city,country,iso_code
0,1458512,Segunda División,707,306,26.03.2021 17:00,0:0,top,Sporting Gijón,Alcorcón,7.0,65.0,17.0,48.0,22.0,82.0,1.95,2.77,4.72,1.33,3.08,None,0.0,0.0,0.0,2.0,Fav,with_events,0,1905,29538,Gijon,Spain,NaN
1,1458506,Segunda División,707,306,26.03.2021 19:00,1:3,top,Castellon,Espanyol,21.0,41.0,1.0,82.0,22.0,82.0,6.06,3.64,1.62,1.61,2.28,13,0.0,0.0,1.0,4.0,Pre,with_events,4,1922,14485,Castellon de la Plana,Spain,NaN
2,1459331,First League,435,306,26.03.2021 11:00,2:0,sec,Mykolaiv,Alians Lypova Dolyna,4.0,53.0,6.0,51.0,16.0,68.0,1.82,3.42,3.80,1.84,1.90,39,0.0,0.0,1.0,8.0,Fav,with_events,2,1920,25175,Mykolajiv,Ukraine,UA


#### Check last file version

In [21]:
with open('pickle_files/cities_countries/df_uniq_cities_coords_created_16_09_2022', 'rb') as f:
    df_uniq_cities_coords = pickle.load(f)

In [22]:
print('shape - ', df_uniq_cities_coords.shape[0])
df_uniq_cities_coords.tail(3)

shape -  8873


,city,iso_code,country,longitude,latitude,lat,lon
8870,Meerbeeke,BE,Belgium,4.040278,50.824444,50n49,4e2
8871,Siheung-si,KR,Korea Republic,126.802887,37.379800,37n22,126e48
8872,Urozhaynaya,BY,Belarus,26.106700,53.678700,53n40,26e6


In [23]:
col_names = ['city', 'iso_code', 'country']
df_for_coords = df_for_coords.merge(df_uniq_cities_coords, how='left', left_on=col_names, right_on=col_names)
print('shape - ', df_for_coords.shape[0])

shape -  403262


In [24]:
df_for_coords.head(3)

,game_id,ligue_header,comp_id,season_id,game_utc,goals,ligue,f_team,s_team,pos_f,pts_f,pos_s,pts_s,max_pos,max_pts,1,X,2,ТМ 2.5,ТБ 2.5,first_g,owngoal,mispen,redcard,ct_cards,f_pow,desc,ct_goals,founded,capacity,city,country,iso_code,longitude,latitude,lat,lon
0,1458512,Segunda División,707,306,26.03.2021 17:00,0:0,top,Sporting Gijón,Alcorcón,7.0,65.0,17.0,48.0,22.0,82.0,1.95,2.77,4.72,1.33,3.08,None,0.0,0.0,0.0,2.0,Fav,with_events,0,1905,29538,Gijon,Spain,NaN,NaN,NaN,NaN,NaN
1,1458506,Segunda División,707,306,26.03.2021 19:00,1:3,top,Castellon,Espanyol,21.0,41.0,1.0,82.0,22.0,82.0,6.06,3.64,1.62,1.61,2.28,13,0.0,0.0,1.0,4.0,Pre,with_events,4,1922,14485,Castellon de la Plana,Spain,NaN,NaN,NaN,NaN,NaN
2,1459331,First League,435,306,26.03.2021 11:00,2:0,sec,Mykolaiv,Alians Lypova Dolyna,4.0,53.0,6.0,51.0,16.0,68.0,1.82,3.42,3.80,1.84,1.90,39,0.0,0.0,1.0,8.0,Fav,with_events,2,1920,25175,Mykolajiv,Ukraine,UA,NaN,NaN,NaN,NaN


In [25]:
print('shape - ', df_for_coords.shape[0])
df_include_coords = df_for_coords[~df_for_coords.longitude.isna()].copy().reset_index(drop=True)
print('shape - ', df_include_coords.shape[0])

shape -  403262
shape -  201427


In [26]:
df_wt_coords = df_for_coords[df_for_coords.longitude.isna()].copy().reset_index(drop=True)
print('shape - ', df_wt_coords.shape)

shape -  (201835, 37)


In [27]:
df_uniq_city_wt_coords = df_wt_coords[['country', 'city', 'iso_code']].groupby(by=['city', 'iso_code']).first().reset_index()
print('shape - ', df_uniq_city_wt_coords.shape)
df_uniq_city_wt_coords.head(3)

shape -  (1331, 3)


,city,iso_code,country
0,Fuente de Cantos,ES,Spain
1,'Akko,IL,Israel
2,Aba,NG,Nigeria


In [28]:
df_uniq_city_wt_coords[df_uniq_city_wt_coords.duplicated(subset=['city', 'iso_code'])]

,city,iso_code,country


In [ ]:
# 2 Stop *****************************************************

#### Get first geo coordinates from CityCoordinator()

In [29]:
# labrary example
c = CityCoordinator()
# c.get_city(city_name="London", country_code_iso="GB")

In [31]:
# df_uniq_city_wt_coords.iso_code.value_counts(dropna=False)

#### If 'df_uniq_city_wt_coords' is empty go to 'Lat_lon' section

In [32]:
df_for_loc = df_uniq_city_wt_coords[['city', 'iso_code']].apply(lambda x: c.get_city(city_name=x[0], country_code_iso=x[1]), axis=1)

In [ ]:
# df_for_loc

In [33]:
location  = [x['location'] if x != None else None for x in df_for_loc]
longitude = [x['lon'] if x != None else None for x in location]
latitude  = [x['lat'] if x != None else None for x in location]

In [34]:
df_uniq_city_wt_coords['longitude'] = longitude
df_uniq_city_wt_coords['latitude']  = latitude

In [35]:
print('shape - ', df_uniq_city_wt_coords.shape[0])
df_uniq_city_wt_coords.head(3)

shape -  1331


,city,iso_code,country,longitude,latitude
0,Fuente de Cantos,ES,Spain,None,None
1,'Akko,IL,Israel,None,None
2,Aba,NG,Nigeria,7.36667,5.10658


In [36]:
df_uniq_city_new = df_uniq_city_wt_coords[~df_uniq_city_wt_coords.longitude.isna()].copy().reset_index(drop=True)
print('shape - ', df_uniq_city_new.shape[0])

shape -  353


In [37]:
df_city_for_geolocator = df_uniq_city_wt_coords[df_uniq_city_wt_coords.longitude.isna()].copy().reset_index(drop=True)
print('shape - ', df_city_for_geolocator.shape[0])

shape -  978


In [38]:
df_city_for_geolocator.head(3)

,city,iso_code,country,longitude,latitude
0,Fuente de Cantos,ES,Spain,None,None
1,'Akko,IL,Israel,None,None
2,Ababa,ET,Ethiopia,None,None


In [39]:
cities_countries = list(df_city_for_geolocator[['city', 'country']].apply(lambda x: (x[0], x[1]), axis=1))

In [40]:
cities_countries[-3:]

[('Žytkavičy', 'Belarus'),
 ('ʿAmmān (Amman)', 'Jordan'),
 ('ẓabī', 'United Arab Emirates')]

In [ ]:
# 2 Stop ************************************************************

#### Parsing coords from geolocator

#### Fix errors in the cities or countries names during parsing:
##### GeocoderUnavailable: Max retries exceeded with url: /search?q=the+city+Odense%2CDenmark

In [92]:
HtmlParser.cities_data_with_geocoordinates(cities_countries[501:], name_saved_file='cities_data_21_10_2022_all')
# All cities data saved with last names: ('Žižkov', 'Czech Republic')

Cities_data_saved 500 - ('ẓabī', 'United Arab Emirates')
All cities data saved with last names: ('ẓabī', 'United Arab Emirates')


#### For fix errors values if it exist

In [94]:
# len(cities_countries)

In [93]:
# cities_countries[500]

In [96]:
# for i, city in enumerate(cities_countries):
#     if 'Chi Minh' in str(city[0]):
#         print(i, city)

In [95]:
# cities_countries[833] = ('Ho Chi Minh', 'Vietnam')

##### *******

In [98]:
## If parser save data in one file start this open file code and don't use remainder_data.
with open('pickle_files/cities_countries/cities_data_21_10_2022/cities_data_21_10_2022_all_Maijdi_Bangladesh', 'rb') as f:
    cities_data_1 = pickle.load(f)
with open('pickle_files/cities_countries/cities_data_21_10_2022/cities_data_21_10_2022_all_ẓabī_United Arab Emirates', 'rb') as f:
    cities_data_2 = pickle.load(f)     

In [100]:
cities_data = cities_data_1 + cities_data_2
len(cities_data)

978

In [103]:
# cities_data

In [101]:
len(cities_countries)

978

#### Remainder_data. This code needs to start until len(remainder_data) == 0, if haven't errors

In [ ]:
# path = 'pickle_files/cities_countries/cities_data_26_03_2021/'
# all_cities_data = []

# for file in os.listdir(path):
#     if file.startswith('cities_data'):
#         with open(path + file, 'rb') as f:
#             cities_data = pickle.load(f)
#             all_cities_data = all_cities_data + cities_data

In [ ]:
# all_cities_data = cities_data 

In [ ]:
# len(all_cities_data)

In [ ]:
# all_unique_data = []

# for city_data in all_cities_data:
#     if city_data not in all_unique_data:
#         all_unique_data.append(city_data)

# len(all_unique_data)        

In [ ]:
# remainder_data = cities_countries.copy()

# for city_data in all_unique_data:
#     remainder_data.remove(city_data[0])
    
# len(remainder_data)    

In [ ]:
# remainder_data

In [ ]:
# HtmlParser.cities_data_with_geocoordinates(remainder_data, name_saved_file='cities_data_26_03_2021')

In [ ]:
# cities_data = all_unique_data.copy()

##### End reminder data ******************************************************

##### Add error parser city's name data deleted beefore if need

In [ ]:
# all_unique_data.insert(-1, [('Ciudad de Florida', 'Uruguay'), None])

In [105]:
len(cities_data)

978

In [104]:
cities_data[-2:]

[[('ʿAmmān (Amman)', 'Jordan'),
  Location(عمان, ناحية عمان, لواء قصبة عمان, عمان, 11110, الأردن, (31.9515694, 35.9239625, 0.0))],
 [('ẓabī', 'United Arab Emirates'),
  Location(أبو ظبي, أبوظبي, أبو ظبي, الإمارات العربية المتحدة, (24.4538352, 54.3774014, 0.0))]]

In [106]:
list_cities, list_countries, list_data = [], [], []

for data in cities_data:
    list_cities.append(data[0][0])
    list_countries.append(data[0][1])
    if data[1] != None:
        list_data.append([data[1].latitude, data[1].longitude])
    else:
        list_data.append([None, None])

In [107]:
list_lat = [x[0] for x in list_data]
list_lon = [x[1] for x in list_data]

In [108]:
df_coord_parsing = pd.DataFrame({'city':list_cities, 'country':list_countries, 'latitude':list_lat, 'longitude':list_lon})

In [ ]:
## 'Colinas-TO' error name 
# df_coord_parsing.head(50)

In [ ]:
# 3 Stop *********************************************************

### Remove coordinates from errors cities names

In [114]:
df_coord_parsing

,city,country,latitude,longitude
0,Fuente de Cantos,Spain,38.246138,-6.307427
1,'Akko,Israel,32.928173,35.075638
2,Ababa,Ethiopia,9.000000,38.750000
3,Abbès,Algeria,29.083202,-2.659006
4,Acarigua-Araure,Venezuela,9.563843,-69.210239
...,...,...,...,...
973,Žabokreky,Slovakia,49.026601,18.919772
974,Ždírec nad Doubravou,Czech Republic,49.695957,15.813602
975,Žytkavičy,Belarus,52.219050,27.857120
976,ʿAmmān (Amman),Jordan,31.951569,35.923963


In [109]:
df_coord_parsing.loc[7]

city            Achnas
country         Cyprus
latitude     35.055549
longitude    33.814098
Name: 7, dtype: object

In [ ]:
# error_inds = [7]
# df_coord_parsing.latitude.loc[error_inds, ]  = None
# df_coord_parsing.longitude.loc[error_inds, ] = None

##### Check columns beefore merge. Column 'iso_code' - it's ok  *******

In [115]:
df_city_for_geolocator

,city,iso_code,country,longitude,latitude
0,Fuente de Cantos,ES,Spain,None,None
1,'Akko,IL,Israel,None,None
2,Ababa,ET,Ethiopia,None,None
3,Abbès,DZ,Algeria,None,None
4,Acarigua-Araure,VE,Venezuela,None,None
...,...,...,...,...,...
973,Žabokreky,SK,Slovakia,None,None
974,Ždírec nad Doubravou,CZ,Czech Republic,None,None
975,Žytkavičy,BY,Belarus,None,None
976,ʿAmmān (Amman),JO,Jordan,None,None


In [116]:
print('shape - df_city_for_geolocator - ', df_city_for_geolocator.shape[0])
print('shape - df_coord_parsing       - ', df_coord_parsing.shape[0])

shape - df_city_for_geolocator -  978
shape - df_coord_parsing       -  978


In [117]:
col_names = ['iso_code', 'city', 'country']
df_coord_parsing = df_coord_parsing.merge(df_city_for_geolocator[col_names], how='left', left_on=['city', 'country'], right_on=['city', 'country'])

In [118]:
df_coord_prs = df_coord_parsing[~df_coord_parsing.latitude.isna()].copy()
print('shape - ', df_coord_prs.shape[0])
df_coord_prs.head(3)

shape -  925


,city,country,latitude,longitude,iso_code
0,Fuente de Cantos,Spain,38.246138,-6.307427,ES
1,'Akko,Israel,32.928173,35.075638,IL
2,Ababa,Ethiopia,9.000000,38.750000,ET


In [119]:
df_uniq_city_new = pd.concat([df_uniq_city_new, df_coord_prs]).reset_index(drop=True)
print('shape - ', df_uniq_city_new.shape[0])

shape -  1278


In [120]:
df_manual_adding = df_coord_parsing[df_coord_parsing.latitude.isna()].copy().reset_index(drop=True)
print('shape - ', df_manual_adding.shape[0])

shape -  53


#### Format fixing errors in cities names: 'error_name' : 'fixing_name'

##### If 'df_manual_adding' is empty - go to 'Lat_lon' section.

In [122]:
# df_manual_adding

### Manual adding cities coordinates and fixing teams data.

##### Latitude - North-South, Longitude - East-West **************

##### Not exist cities coord add 'latitude', 'longitude' - 'na'

In [ ]:
# 4 Stop *********************************************************************

In [124]:
# df_manual_adding.to_csv('csv_files/df_cities_manual_adding_21_10_2022.csv', header=True, index=False)

#### Fix errors in working files: df_wt_coords or df_teams_data_main during fix manual file

In [125]:
print('shape - ', df_wt_coords.shape)
df_wt_coords.head(3)

shape -  (201835, 37)


,game_id,ligue_header,comp_id,season_id,game_utc,goals,ligue,f_team,s_team,pos_f,pts_f,pos_s,pts_s,max_pos,max_pts,1,X,2,ТМ 2.5,ТБ 2.5,first_g,owngoal,mispen,redcard,ct_cards,f_pow,desc,ct_goals,founded,capacity,city,country,iso_code,longitude,latitude,lat,lon
0,1458512,Segunda División,707,306,26.03.2021 17:00,0:0,top,Sporting Gijón,Alcorcón,7.0,65.0,17.0,48.0,22.0,82.0,1.95,2.77,4.72,1.33,3.08,None,0.0,0.0,0.0,2.0,Fav,with_events,0,1905,29538,Gijon,Spain,NaN,NaN,NaN,NaN,NaN
1,1458506,Segunda División,707,306,26.03.2021 19:00,1:3,top,Castellon,Espanyol,21.0,41.0,1.0,82.0,22.0,82.0,6.06,3.64,1.62,1.61,2.28,13,0.0,0.0,1.0,4.0,Pre,with_events,4,1922,14485,Castellon de la Plana,Spain,NaN,NaN,NaN,NaN,NaN
2,1459331,First League,435,306,26.03.2021 11:00,2:0,sec,Mykolaiv,Alians Lypova Dolyna,4.0,53.0,6.0,51.0,16.0,68.0,1.82,3.42,3.80,1.84,1.90,39,0.0,0.0,1.0,8.0,Fav,with_events,2,1920,25175,Mykolajiv,Ukraine,UA,NaN,NaN,NaN,NaN


In [128]:
# Fix some citie's errors names - 'the Eerste divisie', and change it in manual adding file.
# Errors countries names or if one city name has multiple teams - need to change manual in 'df_wt_coords' file and don't use ':' in manual adding file.
# df_wt_coords[df_wt_coords.city == 'Al Khmissat (Khémisset)']
# df_wt_coords[df_wt_coords.f_team == 'Sable Batie']

##### Change error country name for some cities and after that change country name and iso_code in df_for_manual.csv:

In [ ]:
# df_country_code[df_country_code.iso_code == 'NZ']

In [ ]:
# df_country_code[df_country_code.country == 'Uzbekistan']

In [ ]:
# df_wt_coords.loc[5175:5175, 'city'] #= 'São José dos Campos'

In [ ]:
# df_wt_coords.loc[df_wt_coords[df_wt_coords.city == 'Fakkan'].index, 'country']  = 'United Arab Emirates'
# df_wt_coords.loc[df_wt_coords[df_wt_coords.city == 'Fakkan'].index, 'iso_code'] = 'AE'
# df_wt_coords.loc[df_wt_coords[(df_wt_coords.f_team == 'Drean')].index, 'city'] #= 'Dréan' #  & (df_wt_coords.country == 'Spain')
# df_wt_coords.loc[df_wt_coords[df_wt_coords.f_team == 'CSCA'].index, 'country'] #= 'CSCA'
# df_wt_coords.loc[df_wt_coords[df_wt_coords.f_team == 'Lokomotiv Tashkent'].index, 'iso_code'] #= 'UZ'

In [ ]:
# file = open('pickle_files/df_wt_coords_before_manual_add_26_03_2021', 'wb')
# pickle.dump(df_wt_coords, file)  
# file.close()

##### Fix team_data_main file. Check last file's version.

In [ ]:
# df_teams_data_main
# with open('pickle_files/teams_data/df_tmd_main_created_08_08_2022', 'rb') as f:
#     df_teams_data_main = pickle.load(f)

In [ ]:
# df_teams_data_main.shape

In [ ]:
# df_teams_data_main[df_teams_data_main.city == 'Accra']

In [ ]:
# df_teams_data_main.loc[14139].city #= 'Dréan'  

In [ ]:
# df_teams_data_main.drop([12751], inplace=True)
# df_teams_data_main.reset_index(drop=True, inplace=True)

In [ ]:
# df_teams_data_main[df_teams_data_main.team == 'Aigle Noir']

In [ ]:
# df_teams_data_main.loc[14059].country #= 'Australia'

In [ ]:
# file = open('pickle_files/df_tmd_main_created_08_08_2022', 'wb')
# pickle.dump(df_teams_data_main, file)  
# file.close()

#### After manual adding

In [129]:
df_manual_adding = pd.read_csv('csv_files/df_cities_manual_adding_21_10_2022.csv', dtype=str)

In [130]:
df_manual_adding.tail(3)

,city,country,latitude,longitude,iso_code
50,Zubin Potok,Serbia,42.916667,20.683333,RS
51,Ėkibastyz:Ekibastuz,Kazakhstan,51.666667,75.366667,KZ
52,Říše,Czech Republic,49.179722,15.594722,CZ


In [131]:
rename_lst = list(df_manual_adding.city.map(lambda x: x if re.findall(':', str(x)) else None))
rename_lst = [x for x in rename_lst if x != None]
rename_lst = [re.sub(':', '\':\'', x) for x in rename_lst]
str(rename_lst)

'["Al Khmissat (Khémisset)\':\'Khemisset", "As-Sabbah (Chebba)\':\'Chebba", "Bnei-Lod\':\'Lod", "Campsie, Belmore\':\'Campsie", "Castledawson (Caisleán Dhásain)\':\'Castledawson", "Colwyn Bay (Bae Colwyn), Conwy\':\'Colwyn Bay", "Couva, San Fernando\':\'Couva", "Dzyarzhynsk (Dzerzhinsk)\':\'Dzyarzhynsk", "Egilsstaðir, Fljótsdalshérað\':\'Egilsstaðir", "Finchampstead, Berkshire\':\'Finchampstead", "Folād Shahr (Fooladshahr)\':\'Fuladshahr", "Kamenica\':\'Makedonska Kamenica", "Khloraka\':\'Chloraka", "Kinablu\':\'Kota Kinabalu", "Mamurrasi\':\'Mamurras", "Mélina\':\'Melilla", "Naxçıvan (Nakchivan)\':\'Nakchivan", "New Malden, Surrey\':\'New Malden", "Novaci (Novatsi)\':\'Novaci", "Ouessara\':\'Aïn Oussera", "Paty dos Alferes, Rio de Janeiro\':\'Paty dos Alferes", "Pireás\':\'Piraeus", "Qā\'em-Shahr (Ghaemshahr)\':\'Qaem Shahr", "Southport, Merseyside\':\'Southport", "Tziyona\':\'Ness Ziona", "Xinjiyuan\':\'Xi\'an", "Ėkibastyz\':\'Ekibastuz"]'

In [132]:
rename_dict = {}
keys_vals_lst      = []

for x in rename_lst:
    keys_vals = x.split('\':\'')
    keys_vals_lst.append(keys_vals)

for key_val in keys_vals_lst:
    rename_dict[key_val[0]] = key_val[1]

#### Change errors in cities or countries name for df_wt_coords

In [133]:
str(rename_dict)

'{\'Al Khmissat (Khémisset)\': \'Khemisset\', \'As-Sabbah (Chebba)\': \'Chebba\', \'Bnei-Lod\': \'Lod\', \'Campsie, Belmore\': \'Campsie\', \'Castledawson (Caisleán Dhásain)\': \'Castledawson\', \'Colwyn Bay (Bae Colwyn), Conwy\': \'Colwyn Bay\', \'Couva, San Fernando\': \'Couva\', \'Dzyarzhynsk (Dzerzhinsk)\': \'Dzyarzhynsk\', \'Egilsstaðir, Fljótsdalshérað\': \'Egilsstaðir\', \'Finchampstead, Berkshire\': \'Finchampstead\', \'Folād Shahr (Fooladshahr)\': \'Fuladshahr\', \'Kamenica\': \'Makedonska Kamenica\', \'Khloraka\': \'Chloraka\', \'Kinablu\': \'Kota Kinabalu\', \'Mamurrasi\': \'Mamurras\', \'Mélina\': \'Melilla\', \'Naxçıvan (Nakchivan)\': \'Nakchivan\', \'New Malden, Surrey\': \'New Malden\', \'Novaci (Novatsi)\': \'Novaci\', \'Ouessara\': \'Aïn Oussera\', \'Paty dos Alferes, Rio de Janeiro\': \'Paty dos Alferes\', \'Pireás\': \'Piraeus\', "Qā\'em-Shahr (Ghaemshahr)": \'Qaem Shahr\', \'Southport, Merseyside\': \'Southport\', \'Tziyona\': \'Ness Ziona\', \'Xinjiyuan\': "Xi\'an"

In [134]:
df_wt_coords.city = df_wt_coords.city.map(lambda x: rename_dict[x] if x in rename_dict.keys() else x)

In [ ]:
# df_wt_coords[df_wt_coords.city == 'Capriata d\'Orba']

#### Clear manual data

In [135]:
df_manual_adding.city = df_manual_adding.city.map(lambda x: rename_dict[x] if x in rename_dict.keys() else x)
df_manual_adding.city = df_manual_adding.city.map(lambda x: x.split(':')[1] if re.findall(':', x) else x)

In [136]:
df_manual_adding.latitude  = df_manual_adding.latitude.str.strip()
df_manual_adding.longitude = df_manual_adding.longitude.str.strip()

In [137]:
print('shape - ', df_manual_adding.shape[0])
df_manual_adding = df_manual_adding[~df_manual_adding.city.isin(['nd'])].copy().reset_index(drop=True)
print('shape - ', df_manual_adding.shape[0])

shape -  53
shape -  53


In [138]:
print('shape - ', df_uniq_city_new.shape[0])
df_uniq_city_new = pd.concat([df_uniq_city_new, df_manual_adding]).reset_index(drop=True)
print('shape - ', df_uniq_city_new.shape[0])

shape -  1278
shape -  1331


In [139]:
df_uniq_city_new.drop_duplicates(subset=['city', 'iso_code', 'country'], inplace=True, ignore_index=True)
print('shape - ', df_uniq_city_new.shape[0])

shape -  1330


In [140]:
df_uniq_city_new[df_uniq_city_new.longitude.isna()|df_uniq_city_new.latitude.isna()]

,city,iso_code,country,longitude,latitude


In [141]:
df_uniq_city_new = df_uniq_city_new[~df_uniq_city_new.longitude.isna()].copy()
print('shape - ', df_uniq_city_new.shape[0])

shape -  1330


In [142]:
df_uniq_city_new = df_uniq_city_new[df_uniq_city_new.city != 'na'].copy().reset_index(drop=True)
df_uniq_city_new.shape

(1330, 5)

In [ ]:
# 5 Stop *****************************************************

### Lat_lon

In [ ]:
# palmyra = LatLon(Latitude(42.4833333), Longitude(1.4666667)) # Location of Palmyra Atoll in decimal degrees
# palmyra = Lat# Change cities names errors in dataframe:Lon(5.8833, -162.0833) # Same thing but simpler!
# palmyra = LatLon(Latitude(degree = 5, minute = 52, second = 59.88), Longitude(degree = -162, minute = -4.998) # or more complicated!
# cordinates = palmyra.to_string('d% %m% %S% %H') # Print coordinates to degree minute second
# ('5 52 59.88 N', '162 4 59.88 W')

#### If 'df_uniq_city_wt_coords' is empty use in next 3 rows code and go to the end of file if not - don't use LOWER row.

In [ ]:
# df_uniq_city_new = df_include_coords.copy()

In [143]:
df_uniq_city_new['lat'], df_uniq_city_new['lon'] = ParsingDataPrepare.lat_lon_calculate(df_uniq_city_new, col_lat='latitude', col_lon='longitude')

In [144]:
print('shape - ', df_uniq_city_new.shape)
df_uniq_city_new.head(3)

shape -  (1330, 7)


,city,iso_code,country,longitude,latitude,lat,lon
0,Aba,NG,Nigeria,7.36667,5.10658,5n6,7e22
1,Abadan,TM,Turkmenistan,58.19721,38.05415,38n3,58e11
2,Abha,SA,Saudi Arabia,42.50528,18.21639,18n12,42e30


#### ********************************************

#### Concatinate old dataset with new data and if 'df_uniq_city_wt_coords' is not empty:

In [145]:
print('shape - ', df_uniq_cities_coords.shape[0])
df_uniq_cities_coords = pd.concat([df_uniq_cities_coords, df_uniq_city_new]).reset_index(drop=True)
print('shape - ', df_uniq_cities_coords.shape[0])
df_uniq_cities_coords.tail(3)

shape -  8873
shape -  10203


,city,iso_code,country,longitude,latitude,lat,lon
10200,Zubin Potok,RS,Serbia,20.683333,42.916667,42n55,20e40
10201,Ekibastuz,KZ,Kazakhstan,75.366667,51.666667,51n40,75e22
10202,Říše,CZ,Czech Republic,15.594722,49.179722,49n10,15e35


In [146]:
df_uniq_cities_coords.drop_duplicates(subset=['city', 'iso_code', 'country'], inplace=True, ignore_index=True)
print('shape - ', df_uniq_cities_coords.shape)

shape -  (10184, 7)


In [148]:
# file = open('pickle_files/cities_countries/df_uniq_cities_coords_created_21_10_2022', 'wb')
# pickle.dump(df_uniq_cities_coords, file)  
# file.close()

#### Concatinate all dataframes with coordinates

In [149]:
df_wt_coords.drop(columns=['longitude',	'latitude',	'lat', 'lon'], inplace=True)

In [150]:
print('shape - ', df_wt_coords.shape)
cols_merge = ['city', 'iso_code', 'country']
df_coords  = df_wt_coords.merge(df_uniq_city_new, how='left', left_on=cols_merge, right_on=cols_merge)
print('shape - ', df_coords.shape)

shape -  (201835, 33)
shape -  (201835, 37)


In [151]:
print('shape - ', df_coords.shape)
df_coords  = df_coords[(~df_coords.longitude.isna()) & (~df_coords.latitude.isna())].copy()
print('shape - ', df_coords.shape)

shape -  (201835, 37)
shape -  (52265, 37)


In [152]:
print('shape - ', df_coords.shape)
df_coords = df_coords[~df_coords.city.isin(['nd'])].copy().reset_index(drop=True)
print('shape - ', df_coords.shape)

shape -  (52265, 37)
shape -  (52265, 37)


In [153]:
print('shape - ', df_include_coords.shape)
df_include_coords = pd.concat([df_include_coords, df_coords]).reset_index(drop=True)
print('shape - ', df_include_coords.shape)

shape -  (201427, 37)
shape -  (253692, 37)


In [154]:
df_include_coords[(df_include_coords.longitude.isna()) | (df_include_coords.latitude.isna())].values

array([], shape=(0, 37), dtype=object)

#### Check equal cities for each team

In [155]:
df_fteam_cities = df_coords.groupby('f_team').agg({'city':'unique'}).reset_index()

In [163]:
df_fteam_cities.head(3) # city column value: [Sand] - it's ok, don't ok: [Sand, 'Vienna']

,f_team,city
0,2 de Mayo,[Caballero]
1,9715,"[Jamor, Oeiras]"
2,AD Oliveirense,[Oliveira]


In [157]:
len_city_lst = df_fteam_cities.city.map(lambda x: len(x))
max(len_city_lst) # must could == 1

2

In [169]:
# if max(len_city_lst) > 1 need check team for different countries names:
for nb, lt_city in enumerate(df_fteam_cities.city):
    if len(lt_city) > 1:
        print(nb, lt_city)

47 ["Al-'Ayn" 'Al Bāḥa (Al Bahah)']
56 ["Majma'ah" 'ʿAmmān (Amman)']
117 ['Almagro' 'Febrero']
355 ['Bay' 'Colwyn Bay']


In [ ]:
# 6 Stop *****************************************************************

In [ ]:
# ['Juan' 'José'] - team has two cities with different countries

In [170]:
df_fteam_cities.loc[47:47]

,f_team,city
47,Al Ain,"[Al-'Ayn, Al Bāḥa (Al Bahah)]"


In [ ]:
# df_teams_data_main[df_teams_data_main.city == 'Piedras Blancas' ]

In [173]:
# if team name have different countries - it's ok
# df_include_coords[df_include_coords.f_team == 'Al Ain'] 

#### If 'df_uniq_city_wt_coords' is empty use LOWER row:

In [ ]:
# df_include_coords = df_uniq_city_new.copy()

#### ********************

In [174]:
print('shape - ', df_include_coords.shape)
df_include_coords.tail(3)

shape -  (253692, 37)


,game_id,ligue_header,comp_id,season_id,game_utc,goals,ligue,f_team,s_team,pos_f,pts_f,pos_s,pts_s,max_pos,max_pts,1,X,2,ТМ 2.5,ТБ 2.5,first_g,owngoal,mispen,redcard,ct_cards,f_pow,desc,ct_goals,founded,capacity,city,country,iso_code,longitude,latitude,lat,lon
253689,1603935,Championnat National 2,579,310,28.05.2022 16:00,5:0,thr,Fleury 91 FC,Lens II,2.0,65.0,15.0,24.0,16.0,66.0,1.26,5.3,10.00,2.09,1.74,15,0.0,0.0,0.0,0.0,Fav,with_events,5,1970,1000,Merogis,France,FR,2.360931,48.629965,48n37,2e21
253690,1728563,Premier League,623,310,28.05.2022 15:30,2:1,thr,Arbil,Al Sinaah,11.0,43.0,16.0,35.0,20.0,91.0,NaN,NaN,NaN,NaN,NaN,21,0.0,0.0,0.0,0.0,Nodeterm,with_events,3,1968,40000,Arbil,Iraq,IQ,44.009429,36.191165,36n11,44e0
253691,1607740,Landesliga,475,310,28.05.2022 16:00,0:1,thr,St. Jakob,SAK Klagenfurt,9.0,44.0,2.0,54.0,18.0,77.0,2.82,3.3,2.22,2.26,1.56,None,0.0,0.0,0.0,0.0,Pre,with_events,1,1970,1500,Rosental,Austria,AT,14.243774,46.544340,46n32,14e14


In [175]:
df_include_coords.city.value_counts(dropna=False)

Montevideo             1667
Federal                1504
London                 1484
al-Qāhirah             1330
İstanbul               1309
                       ... 
Derventa                  1
Dimitrovgrad              1
Mettersdorf               1
Şımkent (Shymkent)        1
Alphen aan den Rijn       1
Name: city, Length: 5327, dtype: int64

In [177]:
# file = open('pickle_files/cities_countries/df_with_geo_coords_21_10_2022_all', 'wb')
# pickle.dump(df_include_coords, file)  
# file.close()

### Go to astroaspects.ipynb file

In [178]:
# For 'df_with_geo_coords_21_10_2022_all' file aspects don't find (23.08.23) 

##### Example

In [ ]:
# # Exempl
# from  geopy.geocoders import Nominatim
# geolocator = Nominatim(user_agent="Your_Name")

# city ="Russia"
# country ="Perm"
# loc = geolocator.geocode(city+','+ country)
# lat = loc.latitude
# long = loc.longitude

# print(lat, long)